In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType
import os

## DEFINE SENSITIVE VARIABLES
CATALOG_URI = "http://nessie:19120/api/v1"  # Nessie Server URI
WAREHOUSE = "s3://warehouse/"               # Minio Address to Write to
STORAGE_URI = "http://172.22.0.3:9000"      # Minio IP address from docker inspect

# Configure Spark with necessary packages and Iceberg/Nessie settings
conf = (
    pyspark.SparkConf()
        .setAppName('sales_data_app')
        # Include necessary packages
        .set('spark.jars.packages', 'org.postgresql:postgresql:42.7.3,org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.5.0,org.projectnessie.nessie-integrations:nessie-spark-extensions-3.5_2.12:0.77.1,software.amazon.awssdk:bundle:2.24.8,software.amazon.awssdk:url-connection-client:2.24.8')
        # Enable Iceberg and Nessie extensions
        .set('spark.sql.extensions', 'org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions,org.projectnessie.spark.extensions.NessieSparkSessionExtensions')
        # Configure Nessie catalog
        .set('spark.sql.catalog.nessie', 'org.apache.iceberg.spark.SparkCatalog')
        .set('spark.sql.catalog.nessie.uri', CATALOG_URI)
        .set('spark.sql.catalog.nessie.ref', 'main')
        .set('spark.sql.catalog.nessie.authentication.type', 'NONE')
        .set('spark.sql.catalog.nessie.catalog-impl', 'org.apache.iceberg.nessie.NessieCatalog')
        # Set Minio as the S3 endpoint for Iceberg storage
        .set('spark.sql.catalog.nessie.s3.endpoint', STORAGE_URI)
        .set('spark.sql.catalog.nessie.warehouse', WAREHOUSE)
        .set('spark.sql.catalog.nessie.io-impl', 'org.apache.iceberg.aws.s3.S3FileIO')
)

# Start Spark session
spark = SparkSession.builder.config(conf=conf).getOrCreate()
print("Spark Session Started")

# Define a schema for the sales data
schema = StructType([
    StructField("order_id", IntegerType(), True),
    StructField("customer_id", IntegerType(), True),
    StructField("product", StringType(), True),
    StructField("quantity", IntegerType(), True),
    StructField("price", DoubleType(), True),
    StructField("order_date", StringType(), True)
])

# Create a DataFrame with messy sales data (including duplicates and errors)
sales_data = [
    (1, 101, "Laptop", 1, 1000.00, "2023-08-01"),
    (2, 102, "Mouse", 2, 25.50, "2023-08-01"),
    (3, 103, "Keyboard", 1, 45.00, "2023-08-01"),
    (1, 101, "Laptop", 1, 1000.00, "2023-08-01"),  # Duplicate
    (4, 104, "Monitor", None, 200.00, "2023-08-02"),  # Missing quantity
    (5, None, "Mouse", 1, 25.50, "2023-08-02")  # Missing customer_id
]

# Convert the data into a DataFrame
sales_df = spark.createDataFrame(sales_data, schema)

# Create the "sales" namespace
spark.sql("CREATE NAMESPACE IF NOT EXISTS nessie.sales;").show()

# Write the DataFrame to an Iceberg table in the Nessie catalog
sales_df.writeTo("nessie.sales.sales_data_raw").createOrReplace()

# Verify by reading from the Iceberg table
spark.read.table("nessie.sales.sales_data_raw").show()

# Stop the Spark session
# spark.stop()


24/12/22 15:37:54 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


Spark Session Started
++
||
++
++



+--------+-----------+--------+--------+------+----------+
|order_id|customer_id| product|quantity| price|order_date|
+--------+-----------+--------+--------+------+----------+
|       1|        101|  Laptop|       1|1000.0|2023-08-01|
|       2|        102|   Mouse|       2|  25.5|2023-08-01|
|       3|        103|Keyboard|       1|  45.0|2023-08-01|
|       1|        101|  Laptop|       1|1000.0|2023-08-01|
|       4|        104| Monitor|    NULL| 200.0|2023-08-02|
|       5|       NULL|   Mouse|       1|  25.5|2023-08-02|
+--------+-----------+--------+--------+------+----------+



In [2]:
poetry init

SyntaxError: invalid syntax (3644691593.py, line 1)

In [3]:
import yfinance as yf

# Define the stock ticker (e.g., 'AAPL' for Apple)
ticker_symbol = 'AAPL'

# Fetch the ticker data
ticker_data = yf.Ticker(ticker_symbol)

# Get historical market data (e.g., last 30 days)
historical_data = ticker_data.history(period="1mo")

# Get additional information about the company
company_info = ticker_data.info

# Print historical data and company info
print("Historical Data (Last 30 Days):")
print(historical_data)

print("\nCompany Info:")
print(company_info)


ModuleNotFoundError: No module named 'yfinance'

In [12]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit
import yfinance as yf

# Initialize Spark Session
# spark = SparkSession.builder.appName("YahooFinanceData").getOrCreate()

# Input Data: Tickers with their respective start dates
tickers_data = [
    {"ticker": "AAPL", "start_date": "2024-12-10"},
    {"ticker": "MSFT", "start_date": "2024-12-10"},
    {"ticker": "GOOGL", "start_date": "2024-12-10"},
]

# Create RDD from input data
tickers_rdd = spark.sparkContext.parallelize(tickers_data)

# Function to fetch Yahoo Finance data
def fetch_data(params):
    ticker = params["ticker"]
    start_date = params["start_date"]
    try:
        data = yf.download(ticker, start=start_date)
        data.reset_index(inplace=True)
        data["Ticker"] = ticker  # Add a column for ticker
        
        # return data['Ticker']
        # print(data.to_dict("records"))
        return data.to_dict("records")  # Convert to a list of dictionaries
    except Exception as e:
        return [{"Ticker": ticker, "Error": str(e)}]

# Fetch data in parallel
results = tickers_rdd.flatMap(fetch_data).collect()
print(results)

# Convert results to a DataFrame
flattened_results = [item for sublist in results for item in sublist]
df = spark.createDataFrame(flattened_results)

# Show results
df.show(truncate=False)



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


[{('Date', ''): Timestamp('2024-12-10 00:00:00'), ('Adj Close', 'AAPL'): 247.77000427246094, ('Close', 'AAPL'): 247.77000427246094, ('High', 'AAPL'): 248.2100067138672, ('Low', 'AAPL'): 245.33999633789062, ('Open', 'AAPL'): 246.88999938964844, ('Volume', 'AAPL'): 36914800, ('Ticker', ''): 'AAPL'}, {('Date', ''): Timestamp('2024-12-11 00:00:00'), ('Adj Close', 'AAPL'): 246.49000549316406, ('Close', 'AAPL'): 246.49000549316406, ('High', 'AAPL'): 250.8000030517578, ('Low', 'AAPL'): 246.25999450683594, ('Open', 'AAPL'): 247.9600067138672, ('Volume', 'AAPL'): 45205800, ('Ticker', ''): 'AAPL'}, {('Date', ''): Timestamp('2024-12-12 00:00:00'), ('Adj Close', 'AAPL'): 247.9600067138672, ('Close', 'AAPL'): 247.9600067138672, ('High', 'AAPL'): 248.74000549316406, ('Low', 'AAPL'): 245.67999267578125, ('Open', 'AAPL'): 246.88999938964844, ('Volume', 'AAPL'): 32777500, ('Ticker', ''): 'AAPL'}, {('Date', ''): Timestamp('2024-12-13 00:00:00'), ('Adj Close', 'AAPL'): 248.1300048828125, ('Close', 'AAPL'

In [13]:
from pyspark.sql import SparkSession
import yfinance as yf
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, DateType

# Initialize Spark Session
spark = SparkSession.builder.appName("YahooFinanceData").getOrCreate()

# Input Data: List of dictionaries with tickers and start dates
tickers_data = [
    {"ticker": "AAPL", "start_date": "2020-01-01"},
    {"ticker": "MSFT", "start_date": "2019-01-01"},
    {"ticker": "GOOGL", "start_date": "2021-01-01"},
]

# Create RDD from input data
tickers_rdd = spark.sparkContext.parallelize(tickers_data)

# Function to fetch Yahoo Finance data for a single ticker
def fetch_data(params):
    ticker = params["ticker"]
    start_date = params["start_date"]
    try:
        # Fetch data using yfinance
        data = yf.download(ticker, start=start_date, progress=False)
        data.reset_index(inplace=True)  # Reset index to include 'Date' column
        data["Ticker"] = ticker  # Add a column for the ticker symbol
        
        # Convert to a list of tuples
        return [
            (
                row["Date"],
                row["Open"],
                row["High"],
                row["Low"],
                row["Close"],
                row["Adj Close"],
                row["Volume"],
                row["Ticker"],
            )
            for _, row in data.iterrows()
        ]
    except Exception as e:
        return [(None, None, None, None, None, None, None, ticker, str(e))]  # Error handling

# Use flatMap to handle 1-to-many mapping
results_rdd = tickers_rdd.flatMap(fetch_data)

# Define schema for the resulting DataFrame
schema = StructType([
    StructField("Date", DateType(), True),
    StructField("Open", DoubleType(), True),
    StructField("High", DoubleType(), True),
    StructField("Low", DoubleType(), True),
    StructField("Close", DoubleType(), True),
    StructField("Adj Close", DoubleType(), True),
    StructField("Volume", DoubleType(), True),
    StructField("Ticker", StringType(), True),
    StructField("Error", StringType(), True),
])

# Convert RDD to DataFrame
df = spark.createDataFrame(results_rdd, schema=schema)

# Show the results
df.show(truncate=False)


24/12/22 15:49:19 ERROR Executor: Exception in task 0.0 in stage 19.0 (TID 48)3]
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1239, in process
    serializer.dump_stream(out_iter, outfile)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 274, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/opt/spark/python/lib/pyspark.zip/pyspark/util.py", line 83, in wrapper
    return f(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/pyspark/sql/session.py", line 1459, in prepare
    verify_func(obj)
  File "/usr/local/lib/python3.10/dist-packages/pyspark/sql/types.py", line 2201, in verify
    verify_value(obj)
  File "/usr/local/lib/python3.10/dist-packages/pyspark/sql/types.py", line 2164, in verify_struct
    raise PySparkValueError(
pyspa

Py4JJavaError: An error occurred while calling o452.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 19.0 failed 1 times, most recent failure: Lost task 0.0 in stage 19.0 (TID 48) (5967530ddcd3 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1239, in process
    serializer.dump_stream(out_iter, outfile)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 274, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/opt/spark/python/lib/pyspark.zip/pyspark/util.py", line 83, in wrapper
    return f(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/pyspark/sql/session.py", line 1459, in prepare
    verify_func(obj)
  File "/usr/local/lib/python3.10/dist-packages/pyspark/sql/types.py

24/12/22 15:49:20 WARN TaskSetManager: Lost task 1.0 in stage 19.0 (TID 49) (5967530ddcd3 executor driver): TaskKilled (Stage cancelled: Job aborted due to stage failure: Task 0 in stage 19.0 failed 1 times, most recent failure: Lost task 0.0 in stage 19.0 (TID 48) (5967530ddcd3 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1239, in process
    serializer.dump_stream(out_iter, outfile)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 274, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/opt/spark/python/lib/pyspark.zip/pyspark/util.py", line 83, in wrapper
    return f(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/pyspark/sql/session.py", line 1459, in prepare
    verify_func(obj)
  File "/usr/local/lib/python3

In [ ]:
print('heell')

In [1]:
import yfinance

In [2]:
yfinance

<module 'yfinance' from '/usr/local/lib/python3.10/dist-packages/yfinance/__init__.py'>

In [3]:
import requests

In [4]:
requests

<module 'requests' from '/usr/local/lib/python3.10/dist-packages/requests/__init__.py'>

In [17]:
from pyspark.sql import SparkSession
import yfinance as yf
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, DateType

# Initialize Spark Session
spark = SparkSession.builder.appName("YahooFinanceData").getOrCreate()

# Input data: List of dictionaries with tickers and start dates
tickers_data = [
    {"ticker": "AAPL", "start_date": "2020-01-01"},
    {"ticker": "MSFT", "start_date": "2019-01-01"},
    {"ticker": "GOOGL", "start_date": "2021-01-01"},
]

# Create an RDD from the input data
tickers_rdd = spark.sparkContext.parallelize(tickers_data)

# Function to fetch daily data for a ticker
def fetch_data(params):
    ticker = params["ticker"]
    start_date = params["start_date"]
    try:
        # Fetch data using yfinance
        data = yf.download(ticker, start=start_date, progress=False)
        data.reset_index(inplace=True)  # Reset index to include the 'Date' column
        data["Ticker"] = ticker  # Add a column for the ticker symbol
        
        # Extract the required fields: Ticker, Date, Open, Close
        return [
            (row["Ticker"], row["Date"], row["Open"], row["Close"])
            for _, row in data.iterrows()
        ]
    except Exception as e:
        # Return an empty list if an error occurs
        return []

# Use flatMap to fetch data in parallel
results_rdd = tickers_rdd.flatMap(fetch_data)

# Define schema for the resulting DataFrame
schema = StructType([
    StructField("Ticker", StringType(), True),
    StructField("Date", StringType(), True),
    StructField("Open", StringType(), True),
    StructField("Close", StringType(), True),
])

# Convert the RDD to a DataFrame
df = spark.createDataFrame(results_rdd, schema=schema)

# Show the results
df.show()

24/12/22 15:52:32 ERROR Executor: Exception in task 2.0 in stage 27.0 (TID 66)3]
net.razorvine.pickle.PickleException: expected zero arguments for construction of ClassDict (for numpy.core.multiarray._reconstruct). This happens when an unsupported/unregistered class is being unpickled that requires construction arguments. Fix it by registering a custom IObjectConstructor for this class.
	at net.razorvine.pickle.objects.ClassDictConstructor.construct(ClassDictConstructor.java:23)
	at net.razorvine.pickle.Unpickler.load_reduce(Unpickler.java:759)
	at net.razorvine.pickle.Unpickler.dispatch(Unpickler.java:199)
	at net.razorvine.pickle.Unpickler.load(Unpickler.java:109)
	at net.razorvine.pickle.Unpickler.loads(Unpickler.java:122)
	at org.apache.spark.api.python.SerDeUtil$.$anonfun$pythonToJava$2(SerDeUtil.scala:121)
	at scala.collection.Iterator$$anon$11.nextCur(Iterator.scala:486)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:492)
	at scala.collection.Iterator$$anon$10.has

Py4JJavaError: An error occurred while calling o611.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 27.0 failed 1 times, most recent failure: Lost task 2.0 in stage 27.0 (TID 66) (5967530ddcd3 executor driver): net.razorvine.pickle.PickleException: expected zero arguments for construction of ClassDict (for numpy.core.multiarray._reconstruct). This happens when an unsupported/unregistered class is being unpickled that requires construction arguments. Fix it by registering a custom IObjectConstructor for this class.
	at net.razorvine.pickle.objects.ClassDictConstructor.construct(ClassDictConstructor.java:23)
	at net.razorvine.pickle.Unpickler.load_reduce(Unpickler.java:759)
	at net.razorvine.pickle.Unpickler.dispatch(Unpickler.java:199)
	at net.razorvine.pickle.Unpickler.load(Unpickler.java:109)
	at net.razorvine.pickle.Unpickler.loads(Unpickler.java:122)
	at org.apache.spark.api.python.SerDeUtil$.$anonfun$pythonToJava$2(SerDeUtil.scala:12

24/12/22 15:52:33 WARN TaskSetManager: Lost task 0.0 in stage 27.0 (TID 64) (5967530ddcd3 executor driver): TaskKilled (Stage cancelled: Job aborted due to stage failure: Task 2 in stage 27.0 failed 1 times, most recent failure: Lost task 2.0 in stage 27.0 (TID 66) (5967530ddcd3 executor driver): net.razorvine.pickle.PickleException: expected zero arguments for construction of ClassDict (for numpy.core.multiarray._reconstruct). This happens when an unsupported/unregistered class is being unpickled that requires construction arguments. Fix it by registering a custom IObjectConstructor for this class.
	at net.razorvine.pickle.objects.ClassDictConstructor.construct(ClassDictConstructor.java:23)
	at net.razorvine.pickle.Unpickler.load_reduce(Unpickler.java:759)
	at net.razorvine.pickle.Unpickler.dispatch(Unpickler.java:199)
	at net.razorvine.pickle.Unpickler.load(Unpickler.java:109)
	at net.razorvine.pickle.Unpickler.loads(Unpickler.java:122)
	at org.apache.spark.api.python.SerDeUtil$.$ano

In [19]:
from pyspark.sql import SparkSession
import yfinance as yf
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, DateType

# Initialize Spark Session
spark = SparkSession.builder.appName("YahooFinanceData").getOrCreate()

# Input data: List of dictionaries with tickers and start dates
tickers_data = [
    {"ticker": "AAPL", "start_date": "2020-01-01"},
    {"ticker": "MSFT", "start_date": "2019-01-01"},
    {"ticker": "GOOGL", "start_date": "2021-01-01"},
]

# Create an RDD from the input data
tickers_rdd = spark.sparkContext.parallelize(tickers_data)

# Function to fetch daily data for a ticker
def fetch_data(params):
    ticker = params["ticker"]
    start_date = params["start_date"]
    try:
        # Fetch data using yfinance
        data = yf.download(ticker, start=start_date, progress=False)
        
        # Extract only the required fields: Date, Open, Close
        rows = []
        for date, row in data.iterrows():
            rows.append((ticker, date.strftime("%Y-%m-%d"), row["Open"], row["Close"]))
        return rows
    except Exception as e:
        # Log errors as empty list or add optional error logging
        return []

# Use flatMap to fetch data in parallel
results_rdd = tickers_rdd.flatMap(fetch_data)

# Define schema for the resulting DataFrame
schema = StructType([
    StructField("Ticker", StringType(), True),
    StructField("Date", StringType(), True),  # Date as string to avoid serialization issues
    StructField("Open", StringType(), True),
    StructField("Close", StringType(), True),
])

# Convert the RDD to a DataFrame
df = spark.createDataFrame(results_rdd, schema=schema)

# Show the results
df.show(truncate=False)


24/12/22 15:54:48 ERROR Executor: Exception in task 2.0 in stage 31.0 (TID 74)3]
net.razorvine.pickle.PickleException: expected zero arguments for construction of ClassDict (for numpy.core.multiarray._reconstruct). This happens when an unsupported/unregistered class is being unpickled that requires construction arguments. Fix it by registering a custom IObjectConstructor for this class.
	at net.razorvine.pickle.objects.ClassDictConstructor.construct(ClassDictConstructor.java:23)
	at net.razorvine.pickle.Unpickler.load_reduce(Unpickler.java:759)
	at net.razorvine.pickle.Unpickler.dispatch(Unpickler.java:199)
	at net.razorvine.pickle.Unpickler.load(Unpickler.java:109)
	at net.razorvine.pickle.Unpickler.loads(Unpickler.java:122)
	at org.apache.spark.api.python.SerDeUtil$.$anonfun$pythonToJava$2(SerDeUtil.scala:121)
	at scala.collection.Iterator$$anon$11.nextCur(Iterator.scala:486)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:492)
	at scala.collection.Iterator$$anon$10.has

Py4JJavaError: An error occurred while calling o676.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 31.0 failed 1 times, most recent failure: Lost task 2.0 in stage 31.0 (TID 74) (5967530ddcd3 executor driver): net.razorvine.pickle.PickleException: expected zero arguments for construction of ClassDict (for numpy.core.multiarray._reconstruct). This happens when an unsupported/unregistered class is being unpickled that requires construction arguments. Fix it by registering a custom IObjectConstructor for this class.
	at net.razorvine.pickle.objects.ClassDictConstructor.construct(ClassDictConstructor.java:23)
	at net.razorvine.pickle.Unpickler.load_reduce(Unpickler.java:759)
	at net.razorvine.pickle.Unpickler.dispatch(Unpickler.java:199)
	at net.razorvine.pickle.Unpickler.load(Unpickler.java:109)
	at net.razorvine.pickle.Unpickler.loads(Unpickler.java:122)
	at org.apache.spark.api.python.SerDeUtil$.$anonfun$pythonToJava$2(SerDeUtil.scala:12

24/12/22 15:54:49 WARN TaskSetManager: Lost task 0.0 in stage 31.0 (TID 72) (5967530ddcd3 executor driver): TaskKilled (Stage cancelled: Job aborted due to stage failure: Task 2 in stage 31.0 failed 1 times, most recent failure: Lost task 2.0 in stage 31.0 (TID 74) (5967530ddcd3 executor driver): net.razorvine.pickle.PickleException: expected zero arguments for construction of ClassDict (for numpy.core.multiarray._reconstruct). This happens when an unsupported/unregistered class is being unpickled that requires construction arguments. Fix it by registering a custom IObjectConstructor for this class.
	at net.razorvine.pickle.objects.ClassDictConstructor.construct(ClassDictConstructor.java:23)
	at net.razorvine.pickle.Unpickler.load_reduce(Unpickler.java:759)
	at net.razorvine.pickle.Unpickler.dispatch(Unpickler.java:199)
	at net.razorvine.pickle.Unpickler.load(Unpickler.java:109)
	at net.razorvine.pickle.Unpickler.loads(Unpickler.java:122)
	at org.apache.spark.api.python.SerDeUtil$.$ano